### 初始化(导入Spark)

In [1]:
# 显示cell运行时长
%load_ext klab-autotime

In [2]:
import findspark
findspark.init() #找到并激活Spark

import pyspark
from pyspark.sql import SparkSession #SparkSession

from pyspark import SparkContext #SparkContext(类)
#sc = SparkContext("local", "First App") #生成sc实例，和pyspark shell的内置sc一样都是local
sc = SparkContext()
spark = SparkSession(sc)

from pyspark.sql.functions import when,udf
from pyspark.sql.types import StringType, DateType, IntegerType

import numpy as np
import pandas as pd
import os,psutil 
from pyspark.sql.functions import *

time: 3.69 s


In [3]:
print("Spark版本:", sc.version, "\nPython版本:", sc.pythonVer, "\nmaster", sc.master)

Spark版本: 2.4.2 
Python版本: 3.6 
master local[*]
time: 4.29 ms


### 读取数据集

In [4]:
#spark = SparkSession(sc)
df = spark.read.csv("/home/kesci/input/parallel_zwl015100/ticket.csv", header = True, inferSchema = True)
#df = spark.read.csv("/home/kesci/input/parallel_zwl015100/ticket.csv")

time: 1min 12s


In [5]:
df.show(10)

+----+--------+------+---+------------------+----------+----+---+
| _c0|     _c1|   _c2|_c3|               _c4|       _c5| _c6|_c7|
+----+--------+------+---+------------------+----------+----+---+
|null|    null|  null|  9|              null|      null|null|  9|
|null|    null|  null|  9|              null|      null|null|  9|
|8009|20200222|110000|  1|140225199002140040|大同汽车站|浑源|  1|
|8009|20200222|110000|  1|140225199002140040|大同汽车站|浑源|  1|
|8006|20200224| 92000|  1|140225198309220016|大同汽车站|浑源|  1|
|8006|20200224| 92000|  1|140225198309220016|大同汽车站|浑源|  1|
|8010|20200225|104000|  1|140225199206064318|大同汽车站|浑源|  1|
|8010|20200225|104000|  1|140225199306034319|大同汽车站|浑源|  1|
|8010|20200225|104000|  1|140225199206064318|大同汽车站|浑源|  1|
|8010|20200225|104000|  1|140225199306034319|大同汽车站|浑源|  1|
+----+--------+------+---+------------------+----------+----+---+
only showing top 10 rows

time: 253 ms


In [6]:
############暂时不跑##############
SSN = df.select('_c5') #选择出发车站所对应的列
group = SSN.groupby('_c5').count() #将不同车站分组统计
group.show(10) #查看前10个车站及频次
group.count() #一共有4797个出发车站

### 数据整体预处理

In [6]:
#去除完全重复的数据
#df = df.drop_duplicates() 
#筛选列
#df = df.drop('_c3').drop('_c4')
df1 = df.select('_c1')
#对列重命名
df1 = df1.withColumnRenamed("_c1", "departdate")
#new_names = ['schedulecode','departdate','Departtime','startstationname','Reachstationname','seattype']
#df = df.toDF(*new_names)
#删除缺失值
df1 = df1.filter(df1.departdate != 'N')
#展示前十行数据
df1.show(10) 

+----------+
|departdate|
+----------+
|  20200222|
|  20200222|
|  20200224|
|  20200224|
|  20200225|
|  20200225|
|  20200225|
|  20200225|
|  20200226|
|  20200226|
+----------+
only showing top 10 rows

time: 274 ms


In [7]:
############暂时不跑##############
GRD = df1 #选择离站日期所对应的列
group2 = GRD.groupby('departdate').count() #将不同日期分组统计
group2.show(10) #查看前10个日期及频次
group2.count() #一共有xx个日期

### 日期数据预处理

In [7]:
#规范日期数据
#将异常值补全为八位年月日的格式：
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import lit

def fix(str):
    if len(str) < 8:
        str = str + '0'* (8 - len(str))
    elif len(str) > 8:
        str = 'x'
    else:
        pass
    return str

udffix = F.udf(fix)
df1 = df1.withColumn('date_new', udffix(df1.departdate))
df1.show(10)

+----------+--------+
|departdate|date_new|
+----------+--------+
|  20200222|20200222|
|  20200222|20200222|
|  20200224|20200224|
|  20200224|20200224|
|  20200225|20200225|
|  20200225|20200225|
|  20200225|20200225|
|  20200225|20200225|
|  20200226|20200226|
|  20200226|20200226|
+----------+--------+
only showing top 10 rows

time: 1.16 s


In [10]:
############暂时不跑##############
GRD2 = df1 #选择离站日期所对应的列
group3 = GRD2.groupby('date_new').count() #将不同日期分组统计
group3.show(10) #查看前10个日期及频次
group3.count() #一共有xx个日期

In [8]:
#删除非数字形式
#df1 = df1.filter(isdigit(df1.data_new))
#df1 = df1.filter((df1.date_new).isdigit())

#将非数字格式删除：
def dele(str):
    if str.isdigit() == 0:
        #del
        str = '-1'
    else:
        #str = '-1'
        pass
    return str

udfdele = F.udf(dele)
df1 = df1.withColumn('date_new2', udfdele(df1.date_new))
df1.show(10)

+----------+--------+---------+
|departdate|date_new|date_new2|
+----------+--------+---------+
|  20200222|20200222| 20200222|
|  20200222|20200222| 20200222|
|  20200224|20200224| 20200224|
|  20200224|20200224| 20200224|
|  20200225|20200225| 20200225|
|  20200225|20200225| 20200225|
|  20200225|20200225| 20200225|
|  20200225|20200225| 20200225|
|  20200226|20200226| 20200226|
|  20200226|20200226| 20200226|
+----------+--------+---------+
only showing top 10 rows

time: 275 ms


In [9]:
#删除缺失值
df1 = df1.filter(df1.date_new2 != '-1')

time: 41.3 ms


In [13]:
############暂时不跑##############
GRD3 = df1 #选择离站日期所对应的列
group4 = GRD3.groupby('date_new2').count() #将不同日期分组统计
group4.show(10) #查看前10个日期及频次
group4.count() #一共有xx个日期

### 年份分离和统计

In [10]:
#切分日期数据
#添加新列年
def splityear(x):
    return int(x[0:4])
splityear_udf_str = udf(lambda z:splityear(z),StringType())
df1 = df1.withColumn('year',splityear_udf_str(df1['date_new2']))
df1.show(5)

+----------+--------+---------+----+
|departdate|date_new|date_new2|year|
+----------+--------+---------+----+
|  20200222|20200222| 20200222|2020|
|  20200222|20200222| 20200222|2020|
|  20200224|20200224| 20200224|2020|
|  20200224|20200224| 20200224|2020|
|  20200225|20200225| 20200225|2020|
+----------+--------+---------+----+
only showing top 5 rows

time: 400 ms


In [15]:
############暂时不跑##############
GRD4 = df1 #选择离站日期所对应的列
group5 = GRD4.groupby('date_new2').count() #将不同日期分组统计
group5.show(10) #查看前10个日期及频次
group5.count() #一共有xx个日期

In [11]:
df_2020 = df1
df_2021 = df1
df_2020 = df_2020.filter(df_2020.year == 2020)
df_2021 = df_2021.filter(df_2021.year == 2021)

time: 61.4 ms


In [13]:
############暂时不跑##############
#以2020年为示例
GRD5 = df_2020 #选择离站日期所对应的列
group6 = GRD5.groupby('date_new2').count() #将不同日期分组统计
group6.show(10) #查看前10个日期及频次
group6.count() #一共有206个日期

In [ ]:
############暂时不跑##############
#以2021年为示例
df_2021.count()

In [19]:
############暂时不跑##############
GRD5 = df2.select('year') #选择离站年份所对应的列
group6 = GRD5.groupby('year').count() #将不同年份分组统计
group6.show(10) #查看前10个年份及频次
group6.count() #一共有2个年份

### 月份分离和统计

In [12]:
#对于2020年和2021所属数据，分别添加新列月份
def splitmonth(x):
    strm = x[5:6]
    strm = '*' + strm 
    return strm
splitmonth_udf_str = udf(lambda z:splitmonth(z),StringType())
#df1 = df1.withColumn('mon',splitmonth_udf_str(df1['date_new2']))
#df1.show(5)
df_2020 = df_2020.withColumn('mon',splitmonth_udf_str(df_2020['date_new2']))
df_2021 = df_2021.withColumn('mon',splitmonth_udf_str(df_2021['date_new2']))
df_2020.show(5) 
df_2021.show(5) 

+----------+--------+---------+----+---+
|departdate|date_new|date_new2|year|mon|
+----------+--------+---------+----+---+
|  20200222|20200222| 20200222|2020| *2|
|  20200222|20200222| 20200222|2020| *2|
|  20200224|20200224| 20200224|2020| *2|
|  20200224|20200224| 20200224|2020| *2|
|  20200225|20200225| 20200225|2020| *2|
+----------+--------+---------+----+---+
only showing top 5 rows

+----------+--------+---------+----+---+
|departdate|date_new|date_new2|year|mon|
+----------+--------+---------+----+---+
|  20210317|20210317| 20210317|2021| *3|
|  20210317|20210317| 20210317|2021| *3|
|  20210317|20210317| 20210317|2021| *3|
|  20210317|20210317| 20210317|2021| *3|
|  20210317|20210317| 20210317|2021| *3|
+----------+--------+---------+----+---+
only showing top 5 rows

time: 22.6 s


In [26]:
############暂时不跑##############
df2.show()

In [14]:
############暂时不跑##############
#对于2020年按月统计流量
GRD6_1 = df_2020.select('mon') #选择离站月份所对应的列
group7_1 = GRD6_1.groupby('mon').count() #将不同月份分组统计
group7_1.show(12, False) #查看12个月份及频次
group7_1.count() #一共有9个月份？？？

In [17]:
############暂时不跑##############
#对于2021年按月统计流量
GRD6_2 = df_2021.select('month') #选择离站月份所对应的列
group7_2 = GRD6_2.groupby('month').count() #将不同月份分组统计
group7_2.show(12, False) #查看12个月份及频次
group7_2.count() #一共有5个月份？？？

In [18]:
############暂时不跑##############
#计算不同年份的总流量
df_2020.count()
df_2021.count()

In [13]:
#添加新列天
def splitday(x):
    return int(x[6:8])
splitday_udf_str = udf(lambda z:splitday(z),StringType())
#df1 = df1.withColumn('day',splitday_udf_str(df1['date_new2'])) 
#df1.show(5)
#df2 = df2.withColumn('day',splitday_udf_str(df2['date_new2'])) 
#df2.show(5)

time: 2.13 ms


### 2021年各月份

In [14]:
#2021年TOP1月份(3月)
df_2021_3 = df_2021
df_2021_3 = df_2021_3.filter(df_2021_3.mon == '*3')
df_2021_3 = df_2021_3.withColumn('day',splitday_udf_str(df_2021_3['date_new2'])) 
df_2021_3.show(5)

+----------+--------+---------+----+---+---+
|departdate|date_new|date_new2|year|mon|day|
+----------+--------+---------+----+---+---+
|  20210317|20210317| 20210317|2021| *3| 17|
|  20210317|20210317| 20210317|2021| *3| 17|
|  20210317|20210317| 20210317|2021| *3| 17|
|  20210317|20210317| 20210317|2021| *3| 17|
|  20210317|20210317| 20210317|2021| *3| 17|
+----------+--------+---------+----+---+---+
only showing top 5 rows

time: 34.2 s


In [15]:
GRD7_2021_3 = df_2021_3.select('day') #选择离站天所对应的列
group8_2021_3 = GRD7_2021_3.groupby('day').count() #将不同天分组统计
group8_2021_3.show(31, False) #查看不同天及频次
group8_2021_3.count() #一共有29天

+---+------+
|day|count |
+---+------+
|7  |4     |
|15 |43557 |
|11 |600   |
|29 |348959|
|3  |2     |
|30 |408635|
|8  |44218 |
|28 |762153|
|22 |622956|
|16 |65646 |
|5  |5     |
|31 |341443|
|18 |450103|
|27 |629457|
|17 |447530|
|26 |715320|
|6  |3     |
|19 |571246|
|23 |390869|
|25 |433936|
|24 |523032|
|9  |48720 |
|20 |414403|
|10 |2598  |
|12 |55512 |
|13 |47982 |
|21 |549825|
|14 |3735  |
|2  |1     |
+---+------+



29

time: 14min 30s


In [16]:
#2021年TOP2月份(4月)
df_2021_4 = df_2021
df_2021_4 = df_2021_4.filter(df_2021_4.mon == '*4')
df_2021_4 = df_2021_4.withColumn('day',splitday_udf_str(df_2021_4['date_new2'])) 
df_2021_4.show(5)

+----------+--------+---------+----+---+---+
|departdate|date_new|date_new2|year|mon|day|
+----------+--------+---------+----+---+---+
|  20210403|20210403| 20210403|2021| *4|  3|
|  20210402|20210402| 20210402|2021| *4|  2|
|  20210403|20210403| 20210403|2021| *4|  3|
|  20210403|20210403| 20210403|2021| *4|  3|
|  20210403|20210403| 20210403|2021| *4|  3|
+----------+--------+---------+----+---+---+
only showing top 5 rows

time: 28.4 s


In [17]:
GRD7_2021_4 = df_2021_4.select('day') #选择离站天所对应的列
group8_2021_4 = GRD7_2021_4.groupby('day').count() #将不同天分组统计
group8_2021_4.show(31, False) #查看不同天及频次
group8_2021_4.count() #一共有30天

+---+------+
|day|count |
+---+------+
|7  |379690|
|15 |325370|
|11 |278579|
|29 |543   |
|3  |876233|
|30 |4400  |
|8  |334549|
|22 |647   |
|28 |184   |
|16 |38442 |
|5  |795313|
|18 |4448  |
|27 |104   |
|17 |8835  |
|26 |152   |
|6  |484434|
|19 |1706  |
|23 |1078  |
|25 |386   |
|9  |308385|
|24 |697   |
|1  |396728|
|20 |892   |
|10 |303875|
|4  |307416|
|12 |289293|
|13 |356587|
|14 |368263|
|21 |610   |
|2  |696286|
+---+------+



30

time: 14min 29s


In [18]:
#2021年TOP3月份(5月)
df_2021_5 = df_2021
df_2021_5 = df_2021_5.filter(df_2021_5.mon == '*5')
df_2021_5 = df_2021_5.withColumn('day',splitday_udf_str(df_2021_5['date_new2'])) 
df_2021_5.show(5)

+----------+--------+---------+----+---+---+
|departdate|date_new|date_new2|year|mon|day|
+----------+--------+---------+----+---+---+
|  20210504|20210504| 20210504|2021| *5|  4|
|  20210504|20210504| 20210504|2021| *5|  4|
|  20210501|20210501| 20210501|2021| *5|  1|
|  20210501|20210501| 20210501|2021| *5|  1|
|  20210501|20210501| 20210501|2021| *5|  1|
+----------+--------+---------+----+---+---+
only showing top 5 rows

time: 37.2 s


In [19]:
GRD7_2021_5 = df_2021_5.select('day') #选择离站天所对应的列
group8_2021_5 = GRD7_2021_5.groupby('day').count() #将不同天分组统计
group8_2021_5.show(31, False) #查看不同天及频次
group8_2021_5.count() #一共有？天

+---+-----+
|day|count|
+---+-----+
|7  |7    |
|11 |2    |
|3  |125  |
|8  |3    |
|5  |231  |
|6  |11   |
|9  |2    |
|1  |3867 |
|10 |2    |
|4  |124  |
|2  |196  |
+---+-----+



11

time: 14min 14s


In [ ]:
#2021年TOP4月份(2月)
df_2021_2 = df_2021
df_2021_2 = df_2021_2.filter(df_2021_2.mon == '*2')
df_2021_2 = df_2021_2.withColumn('day',splitday_udf_str(df_2021_2['date_new2'])) 
df_2021_2.show(5)

In [ ]:
GRD7_2021_2 = df_2021_2.select('day') #选择离站天所对应的列
group8_2021_2 = GRD7_2021_2.groupby('day').count() #将不同天分组统计
group8_2021_2.show(31, False) #查看不同天及频次
group8_2021_2.count() #一共有？天

In [ ]:
#2021年TOP5月份(1月)
df_2021_1 = df_2021
df_2021_1 = df_2021_1.filter(df_2021_1.mon == '*1')
df_2021_1 = df_2021_1.withColumn('day',splitday_udf_str(df_2021_1['date_new2'])) 
df_2021_1.show(5)

In [ ]:
GRD7_2021_1 = df_2021_1.select('day') #选择离站天所对应的列
group8_2021_1 = GRD7_2021_1.groupby('day').count() #将不同天分组统计
group8_2021_1.show(31, False) #查看不同天及频次
group8_2021_1.count() #一共有？天

### 2020年各月份

In [16]:
#2020年TOP1月份(4月)
df_2020_4 = df_2020
#df_2020_4 = df_2020_4.filter(df_2020_4.month == 4)
#df_2020_4 = df_2020_4.withColumn('day',splitday_udf_str(df_2020_4['date_new2'])) 
df_2020_4.show(5)

In [18]:
df_2020_4 = df_2020_4.filter(df_2020_4.mon == '*4')
df_2020_4.show(5)

In [20]:
df_2020_4 = df_2020_4.withColumn('day',splitday_udf_str(df_2020_4['date_new2'])) 
df_2020_4.show(5)

In [19]:
GRD7_2020_4 = df_2020_4.select('day') #选择离站天所对应的列
group8_2020_4 = GRD7_2020_4.groupby('day').count() #将不同天分组统计
group8_2020_4.show(30, False) #查看不同天及频次
group8_2020_4.count() #一共有30天

In [17]:
#2020年TOP2月份(5月)
df_2020_5 = df_2020
#df_2020_5 = df_2020_5.filter(df_2020_5.month == 5)
#df_2020_5 = df_2020_5.withColumn('day',splitday_udf_str(df_2020_5['date_new2'])) 
df_2020_5.show(5)

In [18]:
df_2020_5 = df_2020_5.filter(df_2020_5.mon == '*5')
df_2020_5.show(5)

In [19]:
df_2020_5 = df_2020_5.withColumn('day',splitday_udf_str(df_2020_5['date_new2'])) 
df_2020_5.show(5)

In [20]:
GRD7_2020_5 = df_2020_5.select('day') #选择离站天所对应的列
group8_2020_5 = GRD7_2020_5.groupby('day').count() #将不同天分组统计
group8_2020_5.show(31, False) #查看不同天及频次
group8_2020_5.count() #一共有？天

In [21]:
#2020年TOP3月份(3月)
df_2020_3 = df_2020
df_2020_3 = df_2020_3.filter(df_2020_3.mon == '*3')
df_2020_3 = df_2020_3.withColumn('day',splitday_udf_str(df_2020_3['date_new2'])) 
df_2020_3.show(5)

In [22]:
GRD7_2020_3 = df_2020_3.select('day') #选择离站天所对应的列
group8_2020_3 = GRD7_2020_3.groupby('day').count() #将不同天分组统计
group8_2020_3.show(31, False) #查看不同天及频次
group8_2020_3.count() #一共有？天

In [23]:
#2020年TOP4月份(6月)
df_2020_6 = df_2020
df_2020_6 = df_2020_6.filter(df_2020_6.mon == '*6')
df_2020_6 = df_2020_6.withColumn('day',splitday_udf_str(df_2020_6['date_new2'])) 
df_2020_6.show(5)

In [24]:
GRD7_2020_6 = df_2020_6.select('day') #选择离站天所对应的列
group8_2020_6 = GRD7_2020_6.groupby('day').count() #将不同天分组统计
group8_2020_6.show(30, False) #查看不同天及频次
group8_2020_6.count() #一共有？天

In [25]:
#2020年TOP5月份(2月)
df_2020_2 = df_2020
df_2020_2 = df_2020_2.filter(df_2020_2.mon == '*2')
df_2020_2 = df_2020_2.withColumn('day',splitday_udf_str(df_2020_2['date_new2'])) 
df_2020_2.show(5)

In [26]:
GRD7_2020_2 = df_2020_2.select('day') #选择离站天所对应的列
group8_2020_2 = GRD7_2020_2.groupby('day').count() #将不同天分组统计
group8_2020_2.show(30, False) #查看不同天及频次
group8_2020_2.count() #一共有？天

In [27]:
#2020年TOP6月份(1月)
df_2020_1 = df_2020
df_2020_1 = df_2020_1.filter(df_2020_1.mon == '*1')
df_2020_1 = df_2020_1.withColumn('day',splitday_udf_str(df_2020_1['date_new2'])) 
df_2020_1.show(5)

In [28]:
GRD7_2020_1 = df_2020_1.select('day') #选择离站天所对应的列
group8_2020_1 = GRD7_2020_1.groupby('day').count() #将不同天分组统计
group8_2020_1.show(31, False) #查看不同天及频次
group8_2020_1.count() #一共有？天

In [29]:
#2020年TOP7月份(8月)
df_2020_8 = df_2020
df_2020_8 = df_2020_8.filter(df_2020_8.mon == '*8')
df_2020_8 = df_2020_8.withColumn('day',splitday_udf_str(df_2020_8['date_new2'])) 
df_2020_8.show(5)

In [30]:
GRD7_2020_8 = df_2020_8.select('day') #选择离站天所对应的列
group8_2020_8 = GRD7_2020_8.groupby('day').count() #将不同天分组统计
group8_2020_8.show(31, False) #查看不同天及频次
group8_2020_8.count() #一共有？天

In [31]:
#2020年TOP8月份(9月)
df_2020_9 = df_2020
df_2020_9 = df_2020_9.filter(df_2020_9.mon == '*9')
df_2020_9 = df_2020_9.withColumn('day',splitday_udf_str(df_2020_9['date_new2'])) 
df_2020_9.show(5)

In [32]:
GRD7_2020_9 = df_2020_9.select('day') #选择离站天所对应的列
group8_2020_9 = GRD7_2020_9.groupby('day').count() #将不同天分组统计
group8_2020_9.show(30, False) #查看不同天及频次
group8_2020_9.count() #一共有？天

In [33]:
#2020年TOP9月份(7月)
df_2020_7 = df_2020
df_2020_7 = df_2020_7.filter(df_2020_7.mon == '*7')
df_2020_7 = df_2020_7.withColumn('day',splitday_udf_str(df_2020_7['date_new2'])) 
df_2020_7.show(5)

In [34]:
GRD7_2020_7 = df_2020_7.select('day') #选择离站天所对应的列
group8_2020_7 = GRD7_2020_7.groupby('day').count() #将不同天分组统计
group8_2020_7.show(31, False) #查看不同天及频次
group8_2020_7.count() #一共有？天

### 绘图（未完成）

In [15]:
#df1 = df1.toPandas()
x = df.groupby('month').count()
x.show(3)
x=x.toPandas()
plt.style.use('fast')

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 生成一张12*4的图
fig = plt.figure(figsize=(12,4))
plt.stackplot(df['month'],
              df['count'] # 面积图是在前一个序列数据基础上，逐层向上累加而构成的图
             ,alpha=0.8)
plt.xlabel('month') # 纵坐标轴标题
plt.ylabel('traffic flowing') # 纵坐标轴标题
plt.title('The Traffic Flowing of Records per month') # 图形标题
plt.legend()
ax = plt.gca()
ax.axes.xaxis.set_ticks([])
plt.show()

In [ ]:
x = x.orderBy(-x['count'])
x.show(5)

### 日期-热力日历图

In [9]:
##########平台自带
import numpy as np
np.random.seed(0)
import seaborn as sns
sns.set() #显示调用，获取默认绘图
uniform_data = np.random.rand(10, 12)
sns.heatmap(uniform_data)

In [18]:
#pip install pyecharts

In [24]:
import seaborn as sns
#np.random.seed(0)
sns.set()

'''
heatmap = HeatMap("车流量出行日历热力图", "xxxx", width=1100)
heatmap.add(
    "",
    group_month,  # 列表数据包含日期以及对应数据
    is_visualmap=True,  # 是否启用视觉映射组件
    visual_range=[0, 5],     # 组件所允许的最大值与最小值
    visual_text_color="#000",   # 组件两端文本颜色
    visual_range_text=["神隐", "微商"],  # 组件两端文本
    visual_range_color=["#D8BFD8", "#FFB6C1", "#EE82EE"],   # 过度颜色，Thistle，LightPink、Violet
    visual_orient="horizontal",     # 组件条的方向，vertical和horizontal可选
    visual_pos="center",    # 组件条距左侧的位置，left、center、right可选，也可用百分数或整数
    visual_top="80%",   # 组件条距离顶部的位置，有top、center、bottom可选，也可用百分数或整数
    # visual_split_number=4,  # 分段型中分段的个数
    # is_piecewise=True,  # 将组件转换为分段型，默认为连续型
    is_calendar_heatmap=True,   # 使用日历热力图
    calendar_cell_size=["auto", 30],    # 单元格大小，默认["auto", 20]
    calendar_date_range=["2018-3-9", "2019-3-10"],   # 日期跨度，可以为str/list
)
heatmap.render('moments.gif')
'''

#返回一个或一组服从“0~1”均匀分布的随机样本值
uniform_data = np.random.rand(10,12) #10个12维数据
sns.heatmap(uniform_data)

In [20]:
df['month'].value_counts()

### 发车日期（按月份）时序图

In [ ]:
library(lubridate)  #R